In [1]:
import pickle
from attention import *
from torch.utils.data import DataLoader
from pytorch_util import RAdam,trainable_parameter
from torch.optim.lr_scheduler import ReduceLROnPlateau
#from pytorch_transformers import AdamW,WarmupLinearSchedule

In [2]:
batch_size = 2048
dim = 512
head_d = 8
head = SimplyInteraction #SimplyHead
encoder_layer,decoder_layer = 4,2
epochs = 10
clip = 0.1
logLoss = True
EncoderLayer=TransformerEncoderLayer
DecoderLayer=TransformerDecoderLayer

dim_feedforward = 1024
dropout = 0.05
lr = 1e-4

In [3]:
with open('../Data/train_data_attention_node.pickle', 'rb') as handle:
    train_node = pickle.load(handle)
with open('../Data/train_data_attention_edge.pickle', 'rb') as handle:
    train_edge = pickle.load(handle)
with open('../Data/train_data_attention_edge_y.pickle', 'rb') as handle:
    train_y = pickle.load(handle)
with open('../Data/train_data_ind.pickle', 'rb') as handle:
    ind = pickle.load(handle)

In [4]:
train_dl = attentionDataset(train_node[:4000000],train_edge[:4000000],ind[:4000000],train_y[:4000000])
train_dl = DataLoader(train_dl,shuffle=True,batch_size=batch_size,collate_fn=collate_fn2,num_workers=4)
val_dl = attentionDataset(train_node[4000000:],train_edge[4000000:],ind[4000000:],train_y[4000000:])
val_dl = DataLoader(val_dl,shuffle=True,batch_size=batch_size,collate_fn=collate_fn2,num_workers=4)

In [5]:
model=Attention3(dim,encoder_layer,head_d,head,EncoderLayer,
                 dropout=dropout,dim_feedforward=dim_feedforward).to('cuda')
paras = trainable_parameter(model)
opt = RAdam(paras,lr=lr,weight_decay=1e-2)
scheduler = ReduceLROnPlateau(opt, 'min',factor=0.5,patience=5,min_lr=1e-04)
model, opt = amp.initialize(model, opt, opt_level="O1")

Selected optimization level O1:  Insert automatic casts around Pytorch functions and Tensor methods.

Defaults for this optimization level are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic
Processing user overrides (additional kwargs that are not None)...
After processing overrides, optimization options are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic


In [6]:
# out,mask,edge,y = next(iter(train_dl))
# out,mask,edge,y = out.to('cuda:0'),mask.to('cuda:0'),edge.to('cuda:0'),y.to('cuda:0')

# opt.zero_grad()
# loss,loss_perType = model(out,mask,edge,y,logLoss)
# loss.backward()

# [(n,p.std()) for n,p in model.named_parameters()]

# [(n,p.grad.mean()) for n,p in model.named_parameters()]

In [10]:
model,bestOpt,bestWeight = train_type2(opt,model,epochs,train_dl,val_dl,paras,clip,\
                                       scheduler=scheduler,logLoss=logLoss)

epoch:0, train_loss: +0.407, val_loss: +0.238, 
train_vector: +1.36|+1.86|+0.23|+0.32|+0.04|+0.15|+0.01|-0.72, 
val_vector  : +1.31|+1.53|-0.09|+0.07|+0.29|-0.01|-0.36|-0.84

epoch:1, train_loss: +0.010, val_loss: -0.071, 
train_vector: +0.99|+1.21|-0.08|-0.08|-0.44|-0.13|-0.35|-1.05, 
val_vector  : +1.03|+1.10|-0.20|-0.30|-0.04|-0.30|-0.78|-1.07

epoch:2, train_loss: -0.197, val_loss: -0.194, 
train_vector: +0.86|+0.75|-0.25|-0.27|-0.64|-0.28|-0.53|-1.21, 
val_vector  : +0.88|+1.11|-0.39|-0.37|-0.32|-0.51|-0.78|-1.16

epoch:3, train_loss: -0.324, val_loss: -0.250, 
train_vector: +0.75|+0.56|-0.37|-0.39|-0.77|-0.40|-0.65|-1.32, 
val_vector  : +1.01|+0.79|-0.45|-0.36|-0.43|-0.57|-0.84|-1.14

epoch:4, train_loss: -0.393, val_loss: -0.209, 
train_vector: +0.70|+0.48|-0.44|-0.46|-0.84|-0.46|-0.72|-1.39, 
val_vector  : +1.13|+1.20|-0.53|-0.37|-0.45|-0.58|-0.88|-1.18

epoch:5, train_loss: -0.450, val_loss: -0.577, 
train_vector: +0.65|+0.44|-0.50|-0.52|-0.91|-0.52|-0.78|-1.45, 
val_vector  :

In [6]:
# O1
model,bestOpt,bestWeight = train_type2(opt,model,epochs,train_dl,val_dl,paras,clip,\
                                       scheduler=scheduler,logLoss=logLoss)

Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 32768.0
epoch:0, train_loss: +0.545, val_loss: +0.128, 
train_vector: +1.60|+1.86|+0.43|+0.45|+0.10|+0.30|+0.22|-0.60, 
val_vector  : +1.25|+1.21|+0.05|-0.09|-0.34|+0.01|-0.38|-0.69

epoch:1, train_loss: -0.053, val_loss: -0.132, 
train_vector: +0.89|+0.92|-0.10|-0.01|-0.51|-0.10|-0.40|-1.11, 
val_vector  : +0.82|+0.75|-0.12|+0.16|-0.45|-0.40|-0.77|-1.05

epoch:2, train_loss: -0.262, val_loss: -0.322, 
train_vector: +0.72|+0.62|-0.32|-0.23|-0.70|-0.29|-0.60|-1.28, 
val_vector  : +0.65|+0.99|-0.55|-0.44|-0.76|-0.55|-0.85|-1.06

Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 65536.0
epoch:3, train_loss: -0.392, val_loss: -0.452, 
train_vector: +0.63|+0.49|-0.46|-0.40|-0.84|-0.43|-0.72|-1.40, 
val_vector  : +0.80|+0.80|-0.56|-0.64|-0.81|-0.74|-1.13|-1.34

Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 65536.0
Gradient overflow.  Skipping step, loss scaler 0 reducing lo

In [8]:
torch.save({
            'epoch': epochs,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': opt.state_dict()
            }, '../Model/attention_edge_512_4_10.tar')

In [6]:
checkpoint = torch.load('../Model/attention_edge_512_4_10.tar')
model.load_state_dict(checkpoint['model_state_dict'])
opt.load_state_dict(checkpoint['optimizer_state_dict'])

In [7]:
# O1 epoch 10 to 20
model,bestOpt,bestWeight = train_type2(opt,model,epochs,train_dl,val_dl,paras,clip,\
                                       scheduler=scheduler,logLoss=logLoss)

Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 32768.0
Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 16384.0
epoch:0, train_loss: -0.805, val_loss: -0.768, 
train_vector: +0.31|+0.06|-0.89|-0.86|-1.32|-0.84|-1.10|-1.80, 
val_vector  : +0.19|+0.46|-1.03|-0.98|-1.11|-0.83|-1.08|-1.77

epoch:1, train_loss: -0.837, val_loss: -0.870, 
train_vector: +0.28|+0.03|-0.92|-0.89|-1.36|-0.87|-1.13|-1.83, 
val_vector  : +0.54|+0.19|-1.10|-0.99|-1.35|-1.02|-1.47|-1.76

Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 16384.0
epoch:2, train_loss: -0.867, val_loss: -0.890, 
train_vector: +0.26|+0.01|-0.95|-0.93|-1.40|-0.90|-1.17|-1.86, 
val_vector  : +0.16|+0.55|-1.08|-0.98|-1.16|-1.20|-1.54|-1.86

Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 16384.0
Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 8192.0
epoch:3, train_loss: -0.897, val_loss: -0.905, 
train_vector: +0.24|-0.02|-0

In [8]:
torch.save({
            'epoch': epochs,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': opt.state_dict()
            }, '../Model/attention_edge_512_4_20.tar')